<a href="https://colab.research.google.com/github/mathew22/tensorflow-tutorial/blob/master/cnn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN on MNIST **data**

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
# read mnist data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True )


# network parameters 
learning_rate = 0.0001
epochs = 10
batch_size = 50


# input data to the network. First dimension is ket as 0 as the batch size 
x = tf.placeholder( tf.float32, [ None, 784] )

# reshaping to 2d single channel. First dimension is -1 indicatinf the batch size
x_shaped = tf.reshape( x, [ -1, 28,28, 1] )

# placeholder for one hot encoded labels
y = tf.placeholder( tf.float32, [ None, 10 ] )

def create_conv_layer( input_data, num_input_channels, num_filters, filter_shape, pool_shape, name ):
    
    conv_filter_shape = [ filter_shape[0], filter_shape[1], num_input_channels, num_filters ]
    
    weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.03), name= name+'_W')
    
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')
    
    # stride =[batch, height, width, channels]
    out_layer = tf.nn.conv2d( input_data, weights, strides=[ 1, 1, 1, 1 ], padding='SAME' )
    
    out_layer += bias
    
    out_layer = tf.nn.relu( out_layer )
    
    # pooling
    ksize = [ 1, pool_shape[0], pool_shape[1], 1]
    strides = [ 1, 2, 2, 1]
    
    out_layer = tf.nn.max_pool( out_layer, ksize=ksize, strides=strides, padding='SAME' )
    
    return out_layer



# convolutional layers
layer_1 = create_conv_layer( x_shaped, 1, 32, [ 5,5] ,[2,2], name='layer1' )
layer_2 = create_conv_layer( layer_1 , 32, 64, [ 5,5] ,[2,2], name='layer2' )

# fully connected layer 
flattened = tf.reshape( layer_2, [-1,7*7*64])


w1 = tf.Variable(tf.truncated_normal( [7*7*64, 1000], stddev=0.03), name='w1' )
b1 = tf.Variable( tf.truncated_normal([1000], stddev=0.01 ), name='b1')
out_1 = tf.matmul( flattened, w1 ) + b1
out_1 = tf.nn.relu( out_1 )

w2 = tf.Variable(tf.truncated_normal( [1000, 10], stddev=0.03), name='w2' )
b2 = tf.Variable( tf.truncated_normal([10],stddev=0.01), name='b2')
out_2 = tf.matmul( out_1, w2 ) + b2
y_cap = tf.nn.relu( out_2 )

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits = out_2 , labels=y ) )

optimizer =tf.train.AdamOptimizer( learning_rate=learning_rate).minimize( cross_entropy )

correct_prediction = tf.equal( tf.argmax( y,1 ), tf.argmax( y_cap, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init_op = tf.global_variables_initializer()



In [0]:
with tf.Session() as sess:
    
    sess.run( init_op )
    total_batch = int( len( mnist.train.labels)/ batch_size)
    for epoch in range( epochs):
        avg_cost = 0
        for i in range(total_batch):        
            batch_x, batch_y = mnist.train.next_batch( batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy ], feed_dict={x:batch_x, y:batch_y })
            avg_cost += c/total_batch
        
        test_acc = sess.run(accuracy,feed_dict={x: mnist.test.images, y: mnist.test.labels})        
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), "test accuracy: {:.3f}".format(test_acc))        
    
    print("\nTraining complete!")
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))    
                           
